# Build a Crew to Tailor Job Applications (Text Input Version)

In [ ]:
# Warning control
import os
import warnings
from dotenv import load_dotenv,find_dotenv

warnings.filterwarnings('ignore')
load_dotenv(find_dotenv())

In [ ]:
from phoenix.otel import register

# configure the Phoenix tracer
tracer_provider = register(
  project_name="my-crewai-vertex-app", # Default is 'default'
  auto_instrument=True # Auto-instrument your app based on installed OI dependencies
)

In [ ]:
# Run gcloud auth application-default login before running this script
# to set up the credentials for the Google Cloud SDK
# and the Google Cloud client libraries.
# SERPER API key is required for this example. 
# You can set it up in your environment variables or directly in the code.


# ... other comments ...
from crewai import LLM
from google.auth import default

credentials, _ = default()

llm = LLM(
    model="vertex_ai/gemini-2.5-flash-preview-05-20",
    temperature=0.7,
    credentials=credentials,
    vertex_project=os.getenv("GOOGLE_CLOUD_PROJECT_ID"),
    vertex_location=os.getenv("GOOGLE_CLOUD_PROJECT_REGION"),
)
# Building a Job Application Assistant with CrewAI

This notebook demonstrates how to build an automated job application assistant using CrewAI. The system takes a job description and personal information as input, then uses a team of specialized AI agents to:

1. Research and analyze job requirements
2. Profile the candidate's skills and experience
3. Create a tailored resume that matches job requirements
4. Prepare for interviews with custom materials

Each agent has a specific role and expertise, working collaboratively to produce the best results.## Environment Setup

This section configures the notebook environment, loads environment variables, and suppresses warnings to ensure a clean execution.# Suppress warnings and load environment variables from .env file
import os
import warnings
from dotenv import load_dotenv, find_dotenv

warnings.filterwarnings('ignore')  # Suppress warning messages
load_dotenv(find_dotenv())  # Load environment variables from .env file## Telemetry Configuration

Configure Phoenix tracer for monitoring and tracking the application's performance.## LLM Configuration

Set up the Language Learning Model (LLM) that will power our agents. This notebook supports:
- Google Vertex AI (Gemini model)
- Groq API (configured to use Llama models)

Before running this section, ensure you have:
1. Set up Google Cloud credentials using `gcloud auth application-default login`
2. Configured necessary API keys in your environment variables## CrewAI Setup

Import the core components from CrewAI that we'll use to build our agent system:
- `Agent`: Represents an AI agent with a specific role, goal, and capabilities
- `Task`: Defines work to be done by an agent, including inputs and expected outputs
- `Crew`: Coordinates multiple agents working together on related tasks## Tools Configuration

Set up specialized tools that our agents will use to gather information and process data:
- `FileReadTool`: Reads the contents of files (used to access the resume)
- `MDXSearchTool`: Performs semantic search on markdown documents
- `SerperDevTool`: Allows agents to search the web for relevant information## Agent Definitions

Each agent in our crew has a specialized role in the job application process:

1. **Researcher**: Analyzes job descriptions to extract key requirements and qualifications
2. **Profiler**: Assesses the candidate's skills and experience against job requirements
3. **Resume Strategist**: Creates a tailored resume that highlights relevant qualifications
4. **Interview Preparer**: Develops materials to help prepare for interviews## Task Definitions

Define the specific tasks that each agent will perform:

1. **Research Task**: Extract and analyze job requirements
2. **Profile Task**: Create a comprehensive candidate profile
3. **Resume Strategy Task**: Develop a tailored resume
4. **Interview Preparation Task**: Create interview preparation materials

Tasks are connected through the `context` parameter, allowing the output of one task to feed into another.## Crew Configuration

Create a crew that coordinates all agents and tasks. The crew manages:
- The sequence of task execution
- Information sharing between agents
- Handling of inputs and outputs## Input Configuration and Execution

Prepare the input data for our job application crew:
1. **Job Description**: The full text of the target job posting
2. **GitHub URL**: Link to the candidate's code portfolio
3. **Personal Writeup**: A description of the candidate's background and qualifications

Then execute the crew to generate tailored application materials.

**Note**: Since LLMs can produce different outputs for the same input, your results may vary from run to run.## Results Visualization

View the final outputs generated by our crew:
1. **Tailored Resume**: A customized resume that emphasizes relevant skills and experience
2. **Interview Materials**: Preparation materials including potential questions and suggested answers
llm.call("Hello there Gemini! How are you doing today?")


os.environ["OPENAI_API_KEY"] = os.environ["GROQ_API_KEY"] 
os.environ["OPENAI_API_BASE"] = os.environ["GROQ_API_BASE"] 
os.environ["OPENAI_MODEL_NAME"] = 'meta-llama/llama-4-scout-17b-16e-instruct'

- Import libraries, APIs and LLM

In [ ]:
from crewai import Agent, Task, Crew

## crewAI Tools

In [ ]:
from crewai_tools import (
  FileReadTool,
  MDXSearchTool,
  SerperDevTool
)

search_tool = SerperDevTool()
read_resume = FileReadTool(file_path='./fake_resume.md')
semantic_search_resume = MDXSearchTool(mdx='./fake_resume.md')

- Uncomment and run the cell below if you wish to view `fake_resume.md` in the notebook.

In [ ]:
from IPython.display import Markdown, display
display(Markdown("./fake_resume.md"))

## Creating Agents

In [ ]:
# Agent 1: Researcher
researcher = Agent(
    role="Tech Job Researcher",
    goal="Make sure to do amazing analysis on "
         "job posting to help job applicants",
    tools = [search_tool],
    verbose=True,
    backstory=(
        "As a Job Researcher, your prowess in "
        "analyzing and extracting critical "
        "information from job descriptions is unmatched."
        "Your skills help pinpoint the necessary "
        "qualifications and skills sought "
        "by employers, forming the foundation for "
        "effective application tailoring."
    ),
    llm=llm
)

In [ ]:
# Agent 2: Profiler
profiler = Agent(
    role="Personal Profiler for Engineers",
    goal="Do incredible research on job applicants "
         "to help them stand out in the job market",
    tools = [search_tool, read_resume, semantic_search_resume],
    verbose=True,
    backstory=(
        "Equipped with analytical prowess, you dissect "
        "and synthesize information "
        "from diverse sources to craft comprehensive "
        "personal and professional profiles, laying the "
        "groundwork for personalized resume enhancements."
    ),
    llm=llm
)

In [ ]:
# Agent 3: Resume Strategist
resume_strategist = Agent(
    role="Resume Strategist for Engineers",
    goal="Find all the best ways to make a "
         "resume stand out in the job market.",
    tools = [search_tool, read_resume, semantic_search_resume],
    verbose=True,
    backstory=(
        "With a strategic mind and an eye for detail, you "
        "excel at refining resumes to highlight the most "
        "relevant skills and experiences, ensuring they "
        "resonate perfectly with the job's requirements."
    ),
    llm=llm
)

In [ ]:
# Agent 4: Interview Preparer
interview_preparer = Agent(
    role="Engineering Interview Preparer",
    goal="Create interview questions and talking points "
         "based on the resume and job requirements",
    tools = [search_tool, read_resume, semantic_search_resume],
    verbose=True,
    backstory=(
        "Your role is crucial in anticipating the dynamics of "
        "interviews. With your ability to formulate key questions "
        "and talking points, you prepare candidates for success, "
        "ensuring they can confidently address all aspects of the "
        "job they are applying for."
    ),
    llm=llm
)

## Creating Tasks

In [ ]:
# Task for Researcher Agent: Extract Job Requirements
research_task = Task(
    description=(
        "Analyze the job description provided ({job_description}) "
        "to extract key skills, experiences, and qualifications "
        "required. Identify and categorize the requirements."
    ),
    expected_output=(
        "A structured list of job requirements, including necessary "
        "skills, qualifications, and experiences."
    ),
    agent=researcher,
)

In [ ]:
# Task for Profiler Agent: Compile Comprehensive Profile
profile_task = Task(
    description=(
        "Compile a detailed personal and professional profile "
        "using the GitHub ({github_url}) URLs, and personal write-up "
        "({personal_writeup}). Utilize tools to extract and "
        "synthesize information from these sources."
    ),
    expected_output=(
        "A comprehensive profile document that includes skills, "
        "project experiences, contributions, interests, and "
        "communication style."
    ),
    agent=profiler,
    async_execution=True
)

- You can pass a list of tasks as `context` to a task.
- The task then takes into account the output of those tasks in its execution.
- The task will not run until it has the output(s) from those tasks.

In [ ]:
# Task for Resume Strategist Agent: Align Resume with Job Requirements
resume_strategy_task = Task(
    description=(
        "Using the profile and job requirements obtained from "
        "previous tasks, tailor the resume to highlight the most "
        "relevant areas. Employ tools to adjust and enhance the "
        "resume content. Make sure this is the best resume even but "
        "don't make up any information. Update every section, "
        "including the initial summary, work experience, skills, "
        "and education. All to better reflect the candidate's "
        "abilities and how it matches the job description."
    ),
    expected_output=(
        "An updated resume that effectively highlights the candidate's "
        "qualifications and experiences relevant to the job."
    ),
    output_file="tailored_resume_text.md",
    context=[research_task, profile_task],
    agent=resume_strategist
)

In [ ]:
# Task for Interview Preparer Agent: Develop Interview Materials
interview_preparation_task = Task(
    description=(
        "Create a set of potential interview questions and talking "
        "points based on the tailored resume and job requirements. "
        "Utilize tools to generate relevant questions and discussion "
        "points. Make sure to use these questions and talking points to "
        "help the candidate highlight the main points of the resume "
        "and how it matches the job description."
    ),
    expected_output=(
        "A document containing key questions and talking points "
        "that the candidate should prepare for the initial interview."
    ),
    output_file="interview_materials_text.md",
    context=[research_task, profile_task, resume_strategy_task],
    agent=interview_preparer
)


## Creating the Crew

In [ ]:
job_application_crew = Crew(
    agents=[researcher,
            profiler,
            resume_strategist,
            interview_preparer],

    tasks=[research_task,
           profile_task,
           resume_strategy_task,
           interview_preparation_task],

    verbose=True
)

## Enter Job Description

In this cell, you can enter the job description as text instead of providing a URL.

In [ ]:
# Enter your job description here
job_description = """ABOUT THE OPPORTUNITY

The AI Foundation is offering a new opportunity to shape our AI initiatives. As the Senior Manager, Responsible AI Solutions, you'll be an individual contributor playing a key role in developing and implementing cutting-edge AI to address complex challenges and create significant impact. Reporting to the Global Director, Business Systems, you will collaborate with staff and consultants to guide the creation and deployment of AI solutions that align with the foundation's values and mission. This is a chance to be a champion for responsible AI, driving innovation and making a real difference.

HOW YOU WILL CONTRIBUTE

The Senior Manager, Responsible AI Solutions will perform the following and/or similar activities:

Develop and implement ethical AI solutions: Collaborate with cross-functional teams to identify and translate business challenges into AI solutions. This involves assessing organizational needs, recommending ethical solutions, gathering requirements, designing and developing AI solutions, and deploying them into production environments.

Manage and oversee AI program: Includes overall program and actions to plan, execute, and deliver AI projects, ensuring they are completed on time, within budget, and meet the required quality standards. This includes defining project scope, gaining insight and agreement from others, allocating resources, managing timelines, and mitigating risks.

Define and track key performance indicators (KPIs): Establish and monitor KPIs to measure the success and impact of AI initiatives, ensuring alignment with business objectives and providing data-driven insights for continuous improvement.

Evaluate and optimize AI model performance: Continuously monitor and evaluate the effectiveness and performance of AI models, identifying areas for improvement and implementing optimization strategies to reduce our carbon footprint and enhance accuracy, efficiency, and scalability.

Ensure compliance with ethical AI practices: Adhere to ethical guidelines and data privacy regulations in all AI-related activities. Promote responsible AI development and deployment within the organization, considering potential biases, fairness, and transparency in AI systems.

Monitor AI advancements: Remain up-to-date with emerging AI trends, technologies, and best practices. Research and recommend new AI tools and techniques to improve operational efficiency and drive innovation.

Communicate effectively with stakeholders: Clearly and effectively communicate project solutions, requirements and updates, technical details, and AI concepts to technical and non-technical audiences, including senior management, business stakeholders, and team members. Translate complex technical information into clear business value for non-technical stakeholders.

WHAT YOU WILL NEED

Proficient in Python and AI frameworks (e.g., TensorFlow, PyTorch), with expertise in machine learning, deep learning, and natural language processing, including Gen AI technologies.

Experienced with Generative AI tools and techniques (e.g., LLMs, LangChain, retrieval-augmented generation) and Agentic architecture.

Familiar with AI Ops principles, including CI/CD pipelines, containerization, and automated testing.

Skilled in optimizing AI workflows (e.g., distributed training, inference optimization, resource utilization - GPU/TPU) through collaboration with AI experts.

Experienced in deploying AI models to production cloud environments (AWS, Azure, GCP), considering performance, cost, and latency.

Adept at monitoring and troubleshooting model performance, addressing issues like performance drift and latency.

Familiar with cloud computing platforms (AWS, Azure, GCP) and data technologies (SQL, Hadoop, Spark) for AI development and deployment.

Demonstrated ability to recommend and apply AI solutions to real-world problems in content-rich domains.

Experienced in working with large datasets and extracting insights from unstructured data.

Strong understanding of content management systems, knowledge graphs, information retrieval techniques and experience with agile development methodologies.

Excellent project management skills, including experience managing complex AI projects.

Ways of working and engaging that align with the Foundation's mission, core values, and commitment to creating a culture of excellence.

SALARY: The starting annualized gross salary is $150,000. The final offer is based on relevant experience and our commitment to internal equity.

LOCATION: This position is based in the foundation's New York office. We operate in a hybrid model and require staff to be in the office three days per week.

EMPLOYMENT TYPE: This is a permanent position.
"""

## Running the Crew

- Set the inputs for the execution of the crew.

In [ ]:
job_application_inputs = {
    'job_description': job_description,
    'github_url': 'https://github.com/padmanabhan-r',
    'personal_writeup': """Padmanabhan is an experienced AI & Data Science Manager with over 10 years of expertise in machine learning, data engineering, and business analytics. He holds a Master's degree in Computer Science with a specialization in Big Data and Data Science from Simon Fraser University, Canada. Currently leading a cross-functional team at Ford Motor Company, he focuses on building scalable ML solutions, Generative AI applications, and intelligent data pipelines that drive business decisions. Padmanabhan has a proven track record of delivering end-to-end AI products, developing predictive models, and creating actionable insights through dashboards and automation. He is passionate about solving complex data problems and thrives in roles that require a blend of technical depth, strategic thinking, and leadership. With strong programming skills in Python, R, and SQL, and hands-on experience with tools like Tableau, Power BI, and TensorFlow, Padmanabhan is well-suited for senior AI/ML roles that demand innovation, ownership, and impact."""
}

**Note**: LLMs can provide different outputs for the same input, so what you get might be different than what you see in the video.

In [ ]:
result = job_application_crew.kickoff(inputs=job_application_inputs)

## View Results

In [ ]:
# Display the tailored resume
display(Markdown("./tailored_resume_text.md"))

In [ ]:
# Display the interview materials
display(Markdown("./interview_materials_text.md"))